# Pre-amble

In [16]:
# loading the required packages
import pandas as pd
import numpy as np
import datetime
import random
import re
import os
import plotnine
from plotnine import *
import plotly.graph_objects as go

# for repeated printouts 
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# to custom displays of row-column df printouts
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

# Data import

Here, we'll load our analysis-ready sentencing data, and print out some of the basic information about the data:

In [17]:
# loading the analysis-ready dataset
intake_analysis = pd.read_csv('../Data/intake_analysis.csv')

# then, we filter only for the black/white defendants 
intake = intake_analysis[(intake_analysis.is_white == True) | 
                            (intake_analysis.is_black == True)].copy()

In [18]:
# printing out basic dataset info
# printing out the basic info
intake.shape
intake.dtypes

(369399, 40)

Unnamed: 0                   int64
CASE_ID                      int64
CASE_PARTICIPANT_ID          int64
RECEIVED_DATE               object
OFFENSE_CATEGORY            object
PARTICIPANT_STATUS          object
AGE_AT_INCIDENT            float64
RACE                        object
GENDER                      object
INCIDENT_CITY               object
INCIDENT_BEGIN_DATE         object
INCIDENT_END_DATE           object
LAW_ENFORCEMENT_AGENCY      object
LAW_ENFORCEMENT_UNIT        object
ARREST_DATE                 object
FELONY_REVIEW_DATE          object
FELONY_REVIEW_RESULT        object
UPDATE_OFFENSE_CATEGORY     object
is_black                    object
is_hispanic                   bool
is_white                    object
is_hisp                    float64
is_female                   object
age_cleaned                float64
felony_review_date          object
felony_review_year         float64
felony_review_month        float64
felony_review_day          float64
felony_review_ym    

In [42]:
# loading the analysis-ready dataset
sentencing_analysis = pd.read_csv('../data/sentencing_analysis.csv')

# we then filter for only black/white defendants
sentencing = sentencing_analysis[(sentencing_analysis.is_black == True) | 
                                    (sentencing_analysis.is_white == True)].copy()

/var/folders/6k/pjvz2kxd4dz2dt0v3l5p885c0000gn/T/ipykernel_51381/171810681.py:2: DtypeWarning:

Columns (10,11,14,25) have mixed types. Specify dtype option on import or set low_memory=False.



In [20]:
# what's the shape of the data? 
print("Data shape: " + str(sentencing_analysis.shape))

Data shape: (186824, 76)


In [21]:
sentencing.dtypes

CASE_ID                                  int64
CASE_PARTICIPANT_ID                      int64
RECEIVED_DATE                           object
OFFENSE_CATEGORY                        object
PRIMARY_CHARGE_FLAG                       bool
CHARGE_ID                                int64
CHARGE_VERSION_ID                        int64
DISPOSITION_CHARGED_OFFENSE_TITLE_x     object
CHARGE_COUNT                             int64
DISPOSITION_DATE                        object
DISPOSITION_CHARGED_CHAPTER             object
DISPOSITION_CHARGED_ACT                 object
DISPOSITION_CHARGED_SECTION             object
DISPOSITION_CHARGED_CLASS               object
DISPOSITION_CHARGED_AOIC                object
CHARGE_DISPOSITION                      object
CHARGE_DISPOSITION_REASON               object
SENTENCE_JUDGE                          object
SENTENCE_COURT_NAME                     object
SENTENCE_COURT_FACILITY                 object
SENTENCE_PHASE                          object
SENTENCE_DATE

## Summary Tables

In [22]:
## recode gender and race for summary stats
intake['Gender'] = np.where(intake.is_female, 'Female', 'Male')
                            
intake['Race'] = np.where(intake.is_black, 'Black',
                         np.where(intake.is_white, 'White', 'Other'))
intake.GENDER.value_counts()
intake.Race.value_counts()

intake.fr_is_rejected.value_counts()

Male                          317418
Female                         51738
Unknown Gender                    16
Unknown                           16
Male name, no gender given        13
Name: GENDER, dtype: int64

Black    303727
White     65672
Name: Race, dtype: int64

False    344890
True      24509
Name: fr_is_rejected, dtype: int64

In [23]:
## get summary stats for different vars in intake and sentencing data

## intake

int_age = pd.DataFrame(intake['age_cleaned'].describe())
int_gender = pd.DataFrame(intake.Gender.value_counts())
int_race = pd.DataFrame(intake.Race.value_counts())
int_reject = pd.DataFrame(intake.fr_is_rejected.value_counts())

int_age.rename(columns = {'age_cleaned':'Age'},inplace = True)
int_reject.rename(columns = {'fr_is_rejected':'Felonies Rejected'},inplace = True)


int_age
int_gender
int_race
int_reject

,Age
count,361098.000000
mean,34.179455
std,12.622652
min,17.000000
25%,24.000000
50%,31.000000
75%,44.000000
max,81.000000


,Gender
Male,317431
Female,51968


,Race
Black,303727
White,65672


,Felonies Rejected
False,344890
True,24509


In [33]:
int_reject.columns


Index(['Felonies Rejected'], dtype='object')

In [24]:
st_labs = ['Count', 'Mean', 'Standard Dev.', 'Minimum', '25th Percentile', 'Median', '75th Percentile', 'Maximum']
st_labs

['Count',
 'Mean',
 'Standard Dev.',
 'Minimum',
 '25th Percentile',
 'Median',
 '75th Percentile',
 'Maximum']

In [35]:
##


## age table
int_age_table = go.Figure(data=[go.Table(
    columnwidth=[1,2],
    header=dict(values=[['<b>Intake - Age</b>']],
                fill_color='ForestGreen',
                line_color='ForestGreen',
                align='right',
                font=dict(color='white')),
    cells=dict(values=[st_labs, int_age.Age],
               fill_color='FloralWhite',
               align='left'))
])
int_age_table.show()

## gender table
int_gender_table = go.Figure(data=[go.Table(
    columnwidth=[1,2],
    header=dict(values=[['<b>Intake - Gender</b>']],
                fill_color='ForestGreen',
                line_color='ForestGreen',
                align='right',
                font=dict(color='white')),
    cells=dict(values=[int_gender.index, int_gender.Gender],
               fill_color='FloralWhite',
               align='left'))
])
int_gender_table.show()

## race table
int_race_table = go.Figure(data=[go.Table(
    columnwidth=[1,2],
    header=dict(values=[['<b>Intake - Race</b>']],
                fill_color='ForestGreen',
                line_color='ForestGreen',
                align='right',
                font=dict(color='white')),
    cells=dict(values=[int_race.index, int_race.Race],
               fill_color='FloralWhite',
               align='left'))
])
int_race_table.show()

## rejected table
int_reject_table = go.Figure(data=[go.Table(
    columnwidth=[1,2],
    header=dict(values=[['<b>Intake - Felonies Rejected</b>']],
                fill_color='ForestGreen',
                line_color='ForestGreen',
                align='right',
                font=dict(color='white')),
    cells=dict(values=[int_reject.index, int_reject['Felonies Rejected']],
               fill_color='FloralWhite',
               align='left'))
])
int_reject_table.show()

In [36]:
## recode gender and race for summary stats
sentencing['Gender'] = np.where(sentencing.is_female, 'Female', 'Male')
sentencing['Race'] = np.where(sentencing.is_black, 'Black',
                         np.where(sentencing.is_white, 'White', 'Other'))
sentencing.GENDER.value_counts()
sentencing.Race.value_counts()

Male                          130067
Female                         20137
Unknown                            7
Male name, no gender given         3
Unknown Gender                     3
Name: GENDER, dtype: int64

Black    122898
White     27347
Name: Race, dtype: int64

In [43]:
sentencing

,CASE_ID,CASE_PARTICIPANT_ID,RECEIVED_DATE,OFFENSE_CATEGORY,PRIMARY_CHARGE_FLAG,CHARGE_ID,CHARGE_VERSION_ID,DISPOSITION_CHARGED_OFFENSE_TITLE_x,CHARGE_COUNT,DISPOSITION_DATE,DISPOSITION_CHARGED_CHAPTER,DISPOSITION_CHARGED_ACT,DISPOSITION_CHARGED_SECTION,DISPOSITION_CHARGED_CLASS,DISPOSITION_CHARGED_AOIC,CHARGE_DISPOSITION,CHARGE_DISPOSITION_REASON,SENTENCE_JUDGE,SENTENCE_COURT_NAME,SENTENCE_COURT_FACILITY,SENTENCE_PHASE,SENTENCE_DATE,SENTENCE_TYPE,CURRENT_SENTENCE_FLAG,COMMITMENT_TYPE,COMMITMENT_TERM,COMMITMENT_UNIT,LENGTH_OF_CASE_in_Days,AGE_AT_INCIDENT,RACE,GENDER,INCIDENT_CITY,INCIDENT_BEGIN_DATE,INCIDENT_END_DATE,LAW_ENFORCEMENT_AGENCY,LAW_ENFORCEMENT_UNIT,ARREST_DATE,FELONY_REVIEW_DATE,FELONY_REVIEW_RESULT,ARRAIGNMENT_DATE,UPDATED_OFFENSE_CATEGORY,is_black,is_hisp,is_white,is_female,age_cleaned,sentence_date,sentence_year,sentence_month,sentence_day,sentence_ym,sa_office_period,sa_timedelta,sa_timedelta_days,BRA_period,BRA_timedelta,BRA_timedelta_days,sentencing_num,sentencing_term_d,sentencing_term_y,is_incarcerated,is_on_probation,regrouped_offense,eligible_offense,DISPOSITION_CHARGED_OFFENSE_TITLE_y,uccs_code,probability,mfj_code,ncrp_code,ncic_code,nibrs_code,charge_desc,offense_category_code,offense_category_desc,offense_type_code,offense_type_desc
0,198055620664,85937621020,08/15/1984 12:00:00 AM,PROMIS Conversion,True,1242194578090,151097726688,FIRST DEGREE MURDER,1,12/17/2014 12:00:00 AM,38,-,9-1(a)(1),X,0000001606,Plea Of Guilty,NaN,Clayton Jay Crane,District 6 - Markham,Markham Courthouse,Amended/Corrected Sentencing,10/16/2014 12:00:00 AM,Prison,True,Illinois Department of Corrections,62.0,Year(s),10982.0,27.0,Black,Male,NaN,08/09/1984 12:00:00 AM,NaN,CHICAGO POLICE DEPT,NaN,08/15/1984 12:00:00 AM,08/15/1984 12:00:00 AM,Charge(S) Approved,09/21/1984 12:00:00 AM,PROMIS Conversion,True,False,False,0.0,27.0,2014-10-16,2014,10,16,2014-10,False,-26,-777,False,-32,-970,62.0,22630.0,62.000000,True,False,PROMIS Conversion,False,FIRST DEGREE MURDER,1010,0.999990,10,010,NaN,09A,Murder,1,Murder,1,Violent
1,198452967058,79292385441,08/23/1984 12:00:00 AM,PROMIS Conversion,True,1249137148269,151678615480,FIRST DEGREE MURDER,1,08/06/2014 12:00:00 AM,38,-,9-1(a)(1),X,0000001606,Verdict Guilty,NaN,Thomas V Gainer,District 1 - Chicago,26TH Street,Resentenced,08/06/2014 12:00:00 AM,Prison,True,Illinois Department of Corrections,1.0,Natural Life,NaN,30.0,Black,Male,NaN,02/10/1983 12:00:00 AM,NaN,CHICAGO POLICE DEPT,NaN,02/10/1983 12:00:00 AM,NaN,NaN,NaN,PROMIS Conversion,True,False,False,0.0,30.0,2014-08-06,2014,8,6,2014-08,False,-28,-848,False,-34,-1041,1.0,25550.0,70.000000,True,False,PROMIS Conversion,False,FIRST DEGREE MURDER,1010,0.999990,10,010,NaN,09A,Murder,1,Murder,1,Violent
3,205272720716,106314583182,01/31/2001 12:00:00 AM,PROMIS Conversion,True,1325561059713,374900949858,POSS AMT CON SUB EXCEPT (A)/(D),1,09/10/2012 12:00:00 AM,720-570\402(C),NaN,NaN,4,0005101110,Plea Of Guilty,NaN,William J Kunkle,District 5 - Bridgeview,Bridgeview Courthouse,Original Sentencing,09/10/2012 12:00:00 AM,Probation,True,710/410 Probation,2.0,Year(s),NaN,33.0,Black,Female,NaN,01/30/2001 12:00:00 AM,NaN,C P D AREA 2 DIST 4,NaN,01/30/2001 02:10:00 AM,NaN,NaN,NaN,PROMIS Conversion,True,False,False,1.0,33.0,2012-09-10,2012,9,10,2012-09,False,-51,-1543,False,-57,-1736,2.0,730.0,2.000000,False,True,PROMIS Conversion,True,POSS AMT CON SUB EXCEPT (A)/(D),3080,0.634869,250,370,3590;3591;3592;3593;3594;3595,NaN,"Distribution, Drug Unspecified",42,Distribution - drug unspecified,3,Drug
4,205411772533,106247326235,02/06/2001 12:00:00 AM,PROMIS Conversion,True,1326510640104,220665098379,DUI LIC SUSPENDED OR REVOKED (EFFECTIVE 4-13-2...,1,09/19/2014 12:00:00 AM,625-5\11-501(C-1)1,NaN,NaN,4,0000012480,Plea Of Guilty,NaN,Timothy Joseph Joyce,District 1 - Chicago,PROMIS,Original Sentencing,09/19/2014 12:00:00 AM,Probation,True,Probation,2.0,Year(s),NaN,49.0,White,Male,NaN,01/14/2001 12:00:00 AM,NaN,C P D AREA 5 DIST 17,NaN,01/14/2001 06:45:00 PM,NaN,NaN,Na

In [46]:


## get summary stats for different vars in sentencing and sentencing data

## sentencing

sent_age = pd.DataFrame(sentencing['age_cleaned'].describe())
sent_gender = pd.DataFrame(sentencing.GENDER.value_counts())
sent_race = pd.DataFrame(sentencing.RACE.value_counts())
sent_incarc = pd.DataFrame(sentencing.is_incarcerated.value_counts())
sent_probat = pd.DataFrame(sentencing.is_on_probation.value_counts())
sent_term = pd.DataFrame(sentencing.sentencing_term_d.describe())

#sent.age_cleaned.rename(columns = {'age_cleaned':'Age'},inplace = True)

sent_age
sent_gender
sent_race
sent_incarc
sent_probat
sent_term

,age_cleaned
count,148328.000000
mean,33.079055
std,12.193864
min,17.000000
25%,23.000000
50%,30.000000
75%,42.000000
max,81.000000


,GENDER
Male,130067
Female,20137
Unknown,7
"Male name, no gender given",3
Unknown Gender,3


,RACE
Black,121886
White,27347
White/Black [Hispanic or Latino],1012


,is_incarcerated
True,80840
False,69405


,is_on_probation
False,92023
True,58222


,sentencing_term_d
count,149221.000000
mean,1115.991079
std,1693.549778
min,0.041667
25%,549.000000
50%,730.000000
75%,1095.000000
max,147825.000000


In [51]:
##


## age table
sent_age_table = go.Figure(data=[go.Table(
    columnwidth=[1,2],
    header=dict(values=[['<b>Sentencing - Age</b>']],
                fill_color='ForestGreen',
                line_color='ForestGreen',
                align='right',
                font=dict(color='white')),
    cells=dict(values=[st_labs, sent_age.age_cleaned],
               fill_color='FloralWhite',
               align='left'))
])
sent_age_table.show()

## gender table
sent_gender_table = go.Figure(data=[go.Table(
    columnwidth=[1,2],
    header=dict(values=[['<b>Sentencing - Gender</b>']],
                fill_color='ForestGreen',
                line_color='ForestGreen',
                align='right',
                font=dict(color='white')),
    cells=dict(values=[sent_gender.index, sent_gender.GENDER],
               fill_color='FloralWhite',
               align='left'))
])
sent_gender_table.show()

## race table
sent_race_table = go.Figure(data=[go.Table(
    columnwidth=[1,2],
    header=dict(values=[['<b>Sentencing - Race</b>']],
                fill_color='ForestGreen',
                line_color='ForestGreen',
                align='right',
                font=dict(color='white')),
    cells=dict(values=[sent_race.index, sent_race.RACE],
               fill_color='FloralWhite',
               align='left'))
])
sent_race_table.show()

In [34]:
## save the stuff

tables = [int_age_table, int_gender_table, int_race_table, 
         sent_age_table, sent_gender_table, sent_race_table]

##int
int_age_table.write_image("../output/int_age_table.png")
int_gender_table.write_image("../output/int_gender_table.png")
int_race_table.write_image("../output/int_race_table.png")

sent_age_table.write_image("../output/sent_age_table.png")
sent_gender_table.write_image("../output/sent_gender_table.png")
sent_race_table.write_image("../output/sent_race_table.png")



